# Transforming a Text file to CEX



## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [3]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source


import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source


## Useful Functions

Save a string to a names file:

In [4]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Convert a Roman Numeral to an Integer:

In [5]:
def fromRoman(s: String) : Int = {
	try {
		val numerals = Map('I' -> 1, 'V' -> 5, 'X' -> 10, 'L' -> 50, 'C' -> 100, 'D' -> 500, 'M' -> 1000)

		s.toUpperCase.map(numerals).foldLeft((0,0)) {
		  case ((sum, last), curr) =>  (sum + curr + (if (last < curr) -2*last else 0), curr) }._1
	} catch {
		case e:Exception => throw new Exception(s""" "${s}" is not a valid Roman Numeral.""")
	}
}

defined function fromRoman

## Load a Template File

Load it into a Vector[String], filtering out any empty lines:

In [6]:
val filePath = s"torq.txt"
val lines: Vector[String] = {
    scala.io.Source.fromFile(filePath).mkString.split("\n").toVector.filter( _.size > 0 )
}

filePath: String = "torq.txt"
lines: Vector[String] = Vector(
  "CAP\u00cdTULO XV. De c\u00f3mo el chichimeca Xolotl, se\u00f1or de las provincias y reinos de Amaqueme, en el septentri\u00f3n o partes del norte, primer poblador de esta Nueva Espa\u00f1a despu\u00e9s de los tultecas, viendo que las gentes que le sol\u00edan hacer guerra ya no parec\u00edan, se determin\u00f3 de entrarles las tierras a buscarlos y envi\u00f3 sus exploradores para que las recorriesen",
  "HACIA LAS PARTES DEL NORTE (en contra de la ciudad de Mexico y en grand\u00edsima distancia apartadas de ella) hubo unas provincias (y puede ser que al presente las haya) cuya principal ciudad fue llamada Amaqueme y cuyos moradores en com\u00fan y gen\u00e9rico vocablo fueron llamados chlchlmecas, gente desnuda de ropas de lana, algod\u00f3n, ni otra cosa que sea de pa\u00f1o o lienzo; pero vestida de pieles de animales; feroces en el aspecto y grandes guerreros, cuyas armas son arcos y flechas. Su sustento ordinario es 

### We need to capture citation valus for Chapters, Paragraphs, and Sentences.

Let's attach an index-number to every line. This will be broadly useful. The result will be a Vector of Tuples: (String, Int). Since that will be confusing, we can create a Class called IndexedLine, and map to a Vector[IndexedLine]:

In [7]:
case class IndexedLine( index: Int, text: String)
val indexedLines: Vector[IndexedLine] = lines.zipWithIndex.map( l => {
    IndexedLine( l._2, l._1 )
})

defined class IndexedLine
indexedLines: Vector[IndexedLine] = Vector(
  IndexedLine(
    0,
    "CAP\u00cdTULO XV. De c\u00f3mo el chichimeca Xolotl, se\u00f1or de las provincias y reinos de Amaqueme, en el septentri\u00f3n o partes del norte, primer poblador de esta Nueva Espa\u00f1a despu\u00e9s de los tultecas, viendo que las gentes que le sol\u00edan hacer guerra ya no parec\u00edan, se determin\u00f3 de entrarles las tierras a buscarlos y envi\u00f3 sus exploradores para que las recorriesen"
  ),
  IndexedLine(
    1,
    "HACIA LAS PARTES DEL NORTE (en contra de la ciudad de Mexico y en grand\u00edsima distancia apartadas de ella) hubo unas provincias (y puede ser que al presente las haya) cuya principal ciudad fue llamada Amaqueme y cuyos moradores en com\u00fan y gen\u00e9rico vocablo fueron llamados chlchlmecas, gente desnuda de ropas de lana, algod\u00f3n, ni otra cosa que sea de pa\u00f1o o lienzo; pero vestida de pieles de animales; feroces en el aspecto y grandes guerreros

Let's get a separate vector of *just* chapter-headings, but keeping their index-numbers from their context in the whole text:

In [8]:
val chapterHeadingStrings: Vector[IndexedLine] = indexedLines.filter( l => {
    l.text.startsWith("CAPÍTULO")
})

chapterHeadingStrings: Vector[IndexedLine] = Vector(
  IndexedLine(
    0,
    "CAP\u00cdTULO XV. De c\u00f3mo el chichimeca Xolotl, se\u00f1or de las provincias y reinos de Amaqueme, en el septentri\u00f3n o partes del norte, primer poblador de esta Nueva Espa\u00f1a despu\u00e9s de los tultecas, viendo que las gentes que le sol\u00edan hacer guerra ya no parec\u00edan, se determin\u00f3 de entrarles las tierras a buscarlos y envi\u00f3 sus exploradores para que las recorriesen"
  ),
  IndexedLine(
    5,
    "CAP\u00cdTULO XVI. De c\u00f3mo el gran chichimeca Xolotl llam\u00f3 a consejo a los grandes de su reino: y de lo que en \u00e9l qued\u00f3 determinado "
  ),
  IndexedLine(
    8,
    "CAP\u00cdTULO XVII. De c\u00f3mo el chichimeca Xolotl, habiendo llegado a estas comarcas de Mexico, despach\u00f3 a su hijo Nopaltzin a buscar las gentes moradoras de la tierra y \u00e9l se volvi\u00f3 a su puesto de Xoloc"
  ),
  IndexedLine(
    10,
    "CAP\u00cdTULO XVIII. De c\u00f3mo volvie

Let's define a ChapterHead class, containing the important data we'll need:

In [9]:
case class ChapterHead(index: Int, label: String, head: String)

defined class ChapterHead

And we map `chapterHeadingStrings` to this new class, by defining a Regular Expression, then applying it to each line in `chapterHeadingStrings`.

In [10]:
val pattern = """CAPÍTULO ([XVI]+)\. (.+)""".r

pattern: scala.util.matching.Regex = CAPÍTULO ([XVI]+)\. (.+)

In [11]:
val chapterHeads: Vector[ChapterHead] = chapterHeadingStrings.map( chs => {
    val pattern(tempLabel, tempText) = chs.text

    val label:String = fromRoman(tempLabel).toString
    val text = tempText
    val index = chs.index
    ChapterHead( index, label, text)
})


chapterHeads: Vector[ChapterHead] = Vector(
  ChapterHead(
    0,
    "15",
    "De c\u00f3mo el chichimeca Xolotl, se\u00f1or de las provincias y reinos de Amaqueme, en el septentri\u00f3n o partes del norte, primer poblador de esta Nueva Espa\u00f1a despu\u00e9s de los tultecas, viendo que las gentes que le sol\u00edan hacer guerra ya no parec\u00edan, se determin\u00f3 de entrarles las tierras a buscarlos y envi\u00f3 sus exploradores para que las recorriesen"
  ),
  ChapterHead(
    5,
    "16",
    "De c\u00f3mo el gran chichimeca Xolotl llam\u00f3 a consejo a los grandes de su reino: y de lo que en \u00e9l qued\u00f3 determinado "
  ),
  ChapterHead(
    8,
    "17",
    "De c\u00f3mo el chichimeca Xolotl, habiendo llegado a estas comarcas de Mexico, despach\u00f3 a su hijo Nopaltzin a buscar las gentes moradoras de la tierra y \u00e9l se volvi\u00f3 a su puesto de Xoloc"
  ),
  ChapterHead(
    10,
    "18",
    "De c\u00f3mo volvieron Nopaltzin y los capitanes exploradores de l

Now we map each ChapterHead to a Vector of IndexedLines that go with that chapter. We can use a map of the indices where a chapter begins and ends.

In [12]:
val chapterMap: Vector[(ChapterHead, Vector[IndexedLine])] = chapterHeads.map( ch => {
    val startIndex: Int = ch.index
    val endIndex: Int = {
       val nextChapter: Vector[ChapterHead] = chapterHeads.filter( _.index > ch.index)
       if ( nextChapter.size == 0 ){ 
           indexedLines.last.index + 1 // end of the whole list
       } else {
           nextChapter.head.index
       }
    }
    val contentLines = indexedLines.filter( il => {
        ( il.index > startIndex ) && ( il.index < endIndex )
    })
    (ch, contentLines )
})

chapterMap: Vector[(ChapterHead, Vector[IndexedLine])] = Vector(
  (
    ChapterHead(
      0,
      "15",
      "De c\u00f3mo el chichimeca Xolotl, se\u00f1or de las provincias y reinos de Amaqueme, en el septentri\u00f3n o partes del norte, primer poblador de esta Nueva Espa\u00f1a despu\u00e9s de los tultecas, viendo que las gentes que le sol\u00edan hacer guerra ya no parec\u00edan, se determin\u00f3 de entrarles las tierras a buscarlos y envi\u00f3 sus exploradores para que las recorriesen"
    ),
    Vector(
      IndexedLine(
        1,
        "HACIA LAS PARTES DEL NORTE (en contra de la ciudad de Mexico y en grand\u00edsima distancia apartadas de ella) hubo unas provincias (y puede ser que al presente las haya) cuya principal ciudad fue llamada Amaqueme y cuyos moradores en com\u00fan y gen\u00e9rico vocablo fueron llamados chlchlmecas, gente desnuda de ropas de lana, algod\u00f3n, ni otra cosa que sea de pa\u00f1o o lienzo; pero vestida de pieles de animales; feroces en el as

We can now build up a CEX file…

In [13]:
val urnBase = CtsUrn("urn:cts:greekLit:torquemada.001.offner:")

urnBase: CtsUrn = CtsUrn("urn:cts:greekLit:torquemada.001.offner:")

In [14]:
val sectionCexVec: Vector[String] = chapterMap.map( cm => {
    val chapt: String = cm._1.label
    val headStr: String = cm._1.head
    val firstLine: String = s"${urnBase}${chapt}.head#${headStr}"
    val sections: Vector[String] = cm._2.zipWithIndex.map( ll => {
        val secNum: String = (ll._2 + 1).toString
        val text: String = ll._1.text
        s"${urnBase}${chapt}.${secNum}#${text}"
    })
    firstLine +: sections
}).flatten

sectionCexVec: Vector[String] = Vector(
  "urn:cts:greekLit:torquemada.001.offner:15.head#De c\u00f3mo el chichimeca Xolotl, se\u00f1or de las provincias y reinos de Amaqueme, en el septentri\u00f3n o partes del norte, primer poblador de esta Nueva Espa\u00f1a despu\u00e9s de los tultecas, viendo que las gentes que le sol\u00edan hacer guerra ya no parec\u00edan, se determin\u00f3 de entrarles las tierras a buscarlos y envi\u00f3 sus exploradores para que las recorriesen",
  "urn:cts:greekLit:torquemada.001.offner:15.1#HACIA LAS PARTES DEL NORTE (en contra de la ciudad de Mexico y en grand\u00edsima distancia apartadas de ella) hubo unas provincias (y puede ser que al presente las haya) cuya principal ciudad fue llamada Amaqueme y cuyos moradores en com\u00fan y gen\u00e9rico vocablo fueron llamados chlchlmecas, gente desnuda de ropas de lana, algod\u00f3n, ni otra cosa que sea de pa\u00f1o o lienzo; pero vestida de pieles de animales; feroces en el aspecto y grandes guerreros, cuyas a

## Make Final CEX File

In [15]:
val cexHeaderPath = s"cex_header.txt"
val headerLines: Vector[String] = {
    scala.io.Source.fromFile(cexHeaderPath).mkString.split("\n").toVector.filter( _.size > 0 )
}
val cexHeader: String = "\n" + headerLines.mkString("\n") + "\n#!ctsdata\n"

cexHeaderPath: String = "cex_header.txt"
headerLines: Vector[String] = Vector(
  "#!cexversion",
  "3.0",
  "#!citelibrary",
  "name#CEX library",
  "urn#urn:cite2:cex:temp_xolotl.v1:temp1",
  "license#CC 3.0 NC-BY",
  "#!ctscatalog",
  "urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang",
  "urn:cts:greekLit:torquemada.001.offner:#chapter/section/sentence#fray Juan de Torquemada#Monarqu\u00eda indiana, 1615#Offner, etc.##true#esp"
)
cexHeader: String = """
#!cexversion
3.0
#!citelibrary
name#CEX library
urn#urn:cite2:cex:temp_xolotl.v1:temp1
license#CC 3.0 NC-BY
#!ctscatalog
urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang
urn:cts:greekLit:torquemada.001.offner:#chapter/section/sentence#fray Juan de Torquemada#Monarquía indiana, 1615#Offner, etc.##true#esp
#!ctsdata
"""

Save it!

In [16]:
val finalCex: String = cexHeader + sectionCexVec.mkString("\n")
val fileName: String = "torq.cex"
saveString( finalCex, "", fileName)

finalCex: String = """
#!cexversion
3.0
#!citelibrary
name#CEX library
urn#urn:cite2:cex:temp_xolotl.v1:temp1
license#CC 3.0 NC-BY
#!ctscatalog
urn#citationScheme#groupName#workTitle#versionLabel#exemplarLabel#online#lang
urn:cts:greekLit:torquemada.001.offner:#chapter/section/sentence#fray Juan de Torquemada#Monarquía indiana, 1615#Offner, etc.##true#esp
#!ctsdata
urn:cts:greekLit:torquemada.001.offner:15.head#De cómo el chichimeca Xolotl, señor de las provincias y reinos de Amaqueme, en el septentrión o partes del norte, primer poblador de esta Nueva España después de los tultecas, viendo que las gentes que le solían hacer guerra ya no parecían, se determinó de entrarles las tierras a buscarlos y envió sus exploradores para que las recorriesen
urn:cts:greekLit:torquemada.001.offner:15.1#HACIA LAS PARTES DEL NORTE (en contra de la ciudad de Mexico y en grandísima distancia apartadas de ella) hubo unas provincias (y puede ser que al presente las haya) cuya principal ciudad fue llamada 